In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os
from sklearn.metrics import accuracy_score

In [3]:
df1=pd.read_excel("/content/drive/MyDrive/case_study1 (1).xlsx")
df2=pd.read_excel("/content/drive/MyDrive/case_study2 (1).xlsx")

In [4]:
df1.isnull().sum()

,0
PROSPECTID,0
Total_TL,0
Tot_Closed_TL,0
Tot_Active_TL,0
Total_TL_opened_L6M,0
Tot_TL_closed_L6M,0
pct_tl_open_L6M,0
pct_tl_closed_L6M,0
pct_active_tl,0
pct_closed_tl,0


In [5]:
df1.shape

(51336, 26)

In [6]:
df1=df1.loc[df1['Age_Oldest_TL']!=-99999]

In [7]:
columns_to_be_removed=[]
for i in df2.columns:
  if df2.loc[df2[i]==-99999].shape[0]>10000:
    columns_to_be_removed.append(i)

In [8]:
df2=df2.drop(columns_to_be_removed,axis=1)

In [9]:
df2.columns

Index(['PROSPECTID', 'time_since_recent_payment', 'num_times_delinquent',
       'max_recent_level_of_deliq', 'num_deliq_6mts', 'num_deliq_12mts',
       'num_deliq_6_12mts', 'num_times_30p_dpd', 'num_times_60p_dpd',
       'num_std', 'num_std_6mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts',
       'num_sub_12mts', 'num_dbt', 'num_dbt_6mts', 'num_dbt_12mts', 'num_lss',
       'num_lss_6mts', 'num_lss_12mts', 'recent_level_of_deliq', 'tot_enq',
       'CC_enq', 'CC_enq_L6m', 'CC_enq_L12m', 'PL_enq', 'PL_enq_L6m',
       'PL_enq_L12m', 'time_since_recent_enq', 'enq_L12m', 'enq_L6m',
       'enq_L3m', 'MARITALSTATUS', 'EDUCATION', 'AGE', 'GENDER',
       'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'pct_of_active_TLs_ever',
       'pct_opened_TLs_L6m_of_L12m', 'pct_currentBal_all_TL', 'CC_Flag',
       'PL_Flag', 'pct_PL_enq_L6m_of_L12m', 'pct_CC_enq_L6m_of_L12m',
       'pct_PL_enq_L6m_of_ever', 'pct_CC_enq_L6m_of_ever', 'HL_Flag',
       'GL_Flag', 'last_prod_enq2', 'first_prod_enq2', '

In [10]:
for i in df2.columns:
  df2=df2.loc[df2[i]!=-99999]


In [11]:
for i in list(df1.columns):
  if i in df2.columns:
    print(i)

PROSPECTID


In [12]:
df=pd.merge(df1,df2,how='inner',left_on=['PROSPECTID'],right_on=['PROSPECTID'])

In [13]:
df

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.00,0.167,0.833,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
42060,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,others,others,702,P1
42061,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,1.0,0.0,1.000,0.0,0,0,ConsumerLoan,others,661,P3
42062,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,others,686,P2


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [15]:
df.shape

(42064, 79)

In [16]:
#we will divide features into two parts i categorical ii numerical

In [17]:
a=[]
for i in df.columns:
  if df[i].dtype=='object':
    a.append(i)

In [18]:
a

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

In [19]:
a.remove('Approved_Flag')

In [20]:
a

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [21]:
#chi2 square test
for i in a:
  chi2,pval,_,_=chi2_contingency(pd.crosstab(df[i],df['Approved_Flag']))
  if pval<0.05:
    print(i,'---',pval)

MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287


In [22]:
#since all categorical features have pval<=0.05,we will accept all

In [23]:
numerical_columns=[]
for i in df.columns:
  if df[i].dtype!='object' and i not in ['PROSPECTID','Approved_Flag']:
    numerical_columns.append(i)

In [24]:
#vif sequentiality check
vif_data=df[numerical_columns]
total_columns=vif_data.shape[1]
column_index=0
columns_to_be_kept=[]

In [25]:
total_columns

72

In [26]:
for i in range(0,total_columns):
  vif_value=variance_inflation_factor(vif_data,column_index)
  print(column_index,'---',vif_value)
  if vif_value<=6:
    columns_to_be_kept.append(numerical_columns[i])
    column_index+=1
  else:
    vif_data=vif_data.drop([numerical_columns[i]],axis=1)

/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 11.320180023967996
0 --- 8.363698035000327
0 --- 6.520647877790928
0 --- 5.149501618212625
1 --- 2.611111040579735


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477228
2 --- 3.8328007921530785
3 --- 6.099653381646723
3 --- 5.581352009642766
4 --- 1.9855843530987776


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 --- inf
5 --- 4.809538302819343
6 --- 23.270628983464636
6 --- 30.595522588100053
6 --- 4.3843464059655854
7 --- 3.064658415523423
8 --- 2.898639771299253
9 --- 4.377876915347322
10 --- 2.207853583695844
11 --- 4.916914200506861
12 --- 5.214702030064725
13 --- 3.3861625024231476
14 --- 7.840583309478997
14 --- 5.255034641721438


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 --- inf
15 --- 7.380634506427238
15 --- 1.4210050015175735
16 --- 8.083255010190316
16 --- 1.624122752404011
17 --- 7.257811920140003
17 --- 15.59624383268298
17 --- 1.8258570471324318
18 --- 1.5080839450032664
19 --- 2.1720888348245753
20 --- 2.623397553527229
21 --- 2.2959970812106167
22 --- 7.360578319196439
22 --- 2.1602387773102554
23 --- 2.8686288267891444
24 --- 6.458218003637272
24 --- 2.847411886563824
25 --- 4.75319815628408
26 --- 16.22735475594825
26 --- 6.424377256363877
26 --- 8.887080381808678
26 --- 2.3804746142952666
27 --- 8.609513476514548
27 --- 13.06755093547673
27 --- 3.500040056654653
28 --- 1.9087955874813773
29 --- 17.006562234161628
29 --- 10.730485153719197
29 --- 2.3538497522950275
30 --- 22.104855915136433
30 --- 2.7971639638512915
31 --- 3.424171203217697
32 --- 10.175021454450922
32 --- 6.408710354561292
32 --- 1.001151196262563
33 --- 3.069197305397274
34 --- 2.8091261600643707
35 --- 20.249538381980678
35 --- 15.864576541593745
35 --- 1.8331649740532

In [27]:
!pip install scipy

In [28]:
from scipy.stats import f_oneway
columns_to_be_kept_numerical=[]
for i in columns_to_be_kept:
  a=list(df[i])
  b=list(df['Approved_Flag'])
  group_p1=[value for value,group in zip(a,b) if group=='P1']
  group_p2=[value for value,group in zip(a,b) if group=='P2']
  group_p3=[value for value,group in zip(a,b) if group=='P3']
  group_p4=[value for value,group in zip(a,b) if group=='P4']

  f_statistic,p_value=f_oneway(group_p1,group_p2,group_p3,group_p4)

  if p_value<=0.05:
    columns_to_be_kept_numerical.append(i)

In [29]:
group_p1

[0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,


In [30]:
df.loc[df['EDUCATION']=='SSC']=1
df.loc[df['EDUCATION']=='12TH']=2
df.loc[df['EDUCATION']=='GRADUATE']=3
df.loc[df['EDUCATION']=='UNDER GRADUATE']=4
df.loc[df['EDUCATION']=='POST-GRADUATE']=5
df.loc[df['EDUCATION']=='OTHERS']=6
df.loc[df['EDUCATION']=='PROFESSIONAL']=7

In [31]:
df

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,2,2,2,2,2,2,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2,2,2,2,2,2
1,3,3,3,3,3,3,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3,3,3,3,3,3
2,1,1,1,1,1,1,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
3,5,5,5,5,5,5,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5,5,5,5,5,5
4,2,2,2,2,2,2,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,2,2,2,2,2,2,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2,2,2,2,2,2
42060,1,1,1,1,1,1,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
42061,1,1,1,1,1,1,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
42062,4,4,4,4,4,4,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4,4,4,4,4,4


In [32]:
df['EDUCATION']

,EDUCATION
0,2
1,3
2,1
3,5
4,2
...,...
42059,2
42060,1
42061,1
42062,4


In [33]:
df_encoded=pd.get_dummies(df,columns=['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2'])

In [34]:
df_encoded.head(1)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,last_prod_enq2_5,last_prod_enq2_6,last_prod_enq2_7,first_prod_enq2_1,first_prod_enq2_2,first_prod_enq2_3,first_prod_enq2_4,first_prod_enq2_5,first_prod_enq2_6,first_prod_enq2_7
0,2,2,2,2,2,2,2.0,2.0,2.0,2.0,...,False,False,False,False,True,False,False,False,False,False


In [35]:
if "Approved_Flag" in df_encoded.columns:
  print("true")
else:
  print("false")

true


In [36]:
df_encoded["Approved_Flag"]

,Approved_Flag
0,2
1,3
2,1
3,5
4,2
...,...
42059,2
42060,1
42061,1
42062,4


In [37]:
x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag'].astype('category').cat.codes

In [38]:
y

,0
0,1
1,2
2,0
3,4
4,1
...,...
42059,1
42060,0
42061,0
42062,3


In [39]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [40]:
y_train

,0
29847,2
32223,1
28558,1
33720,1
41167,4
...,...
6265,2
11284,1
38158,5
860,0


In [41]:
rf_classifier=RandomForestClassifier(n_estimators=200,random_state=42)

In [42]:
rf_classifier.fit(x_train,y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [43]:
print(y_train)

29847    2
32223    1
28558    1
33720    1
41167    4
        ..
6265     2
11284    1
38158    5
860      0
15795    2
Length: 33651, dtype: int8


In [44]:
rf_classifier.fit(x_train,y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [45]:
y_pred=rf_classifier.predict(x_test)

In [46]:
y_pred

array([3, 2, 0, ..., 2, 3, 2], dtype=int8)

In [48]:
accuracy_score(y_test,y_pred)

1.0